In [ ]:
# 필요한 패키지/모듈 가져오기
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
%matplotlib inline

from sklearn import preprocessing
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/컨디션난조,분위기고조/실습과제/5주차/

/content/gdrive/Shareddrives/컨디션난조,분위기고조/실습과제/5주차


In [ ]:
%ls

'4_CBR(knn)_AR_CF.ipynb'
 Admission_Predict.csv
 anime.csv
 AR_CF_anime.csv
 books.csv
 mooc.csv
 rating_complete_anime.csv
 ratings_bk.csv
'[컨디션난조 분위기고조] week5_AR_CF_연재.ipynb'
'[컨디션난조 분위기고조] week5_AR_CF_수진.ipynb'
'[컨디션난조 분위기고조] week5_AR_CF.ipynb'
'[컨디션난조 분위기고조] week5_AR_CF_pre.ipynb'
'[컨디션난조 분위기고조] week5_KNN_AR_CF_보고서.docx'
'[컨디션난조 분위기고조] week5_KNN_회귀.ipynb'
'[컨디션난조 분위기고조] week5_KNN_가람_분류.ipynb'


# Data & library import

In [ ]:
bread = pd.read_csv('./[컨디션난조 분위기고조] week5_AR_데이터.csv')

# Description

In [ ]:
display(bread.head())

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


In [ ]:
bread.shape

(20507, 5)

In [ ]:
bread.dtypes

Transaction         int64
Item               object
date_time          object
period_day         object
weekday_weekend    object
dtype: object

In [ ]:
bread.period_day.unique()

array(['morning', 'afternoon', 'evening', 'night'], dtype=object)

data에서 Null값을 확인할 수 없었다.

In [ ]:
bread.isnull().sum()

Transaction        0
Item               0
date_time          0
period_day         0
weekday_weekend    0
dtype: int64

필요한 column인 transaction과 item을 제외하고 나머지는 drop한다.

In [ ]:
data = bread[['Transaction','Item']]

구매 transaction수와 구매 대상의 종류는 아래와 같다.

In [ ]:
print("Transaction 수 : {}\n구매 품목 종류 수 : {}".format(len(data.Transaction.unique()), len(data.Item.unique())))

Transaction 수 : 9465
구매 품목 종류 수 : 94


# AR (Association Rules; 연관규칙)

mlxtend는 머신러닝에 사용하는 알고리즘들을 제공하는 파이썬 라이브러리로, 연관규칙 분석 시 사용하는 함수도 포함한다. 연관규칙 분석을 진행하기 위해서는 , transaction별 특정 품목을 포함하고 있는지 여부를 True, False로 나타내는 테이블을 만들어야한다. 이를 위해 먼저  Transaction당 구매한 품목이 들어있는 하나의  item list를 생성한다. 이 때, apriori 알고리즘의 정의에 의해, 중복된 연관 규칙은 제거해야 한다.

In [ ]:
df_tmp = data.groupby('Transaction')['Item'].apply(set).apply(list).to_list()
print(df_tmp[:10])

[['Bread'], ['Scandinavian'], ['Cookies', 'Jam', 'Hot chocolate'], ['Muffin'], ['Bread', 'Pastry', 'Coffee'], ['Muffin', 'Pastry', 'Medialuna'], ['Coffee', 'Pastry', 'Medialuna', 'Tea'], ['Pastry', 'Bread'], ['Muffin', 'Bread'], ['Scandinavian', 'Medialuna']]


다음으로는 위와 같은 nested list를 association rule을 찾기 위한 dataframe의 형태로 변경한다.  mlxtend에서 제공하는 TransactionEncoder을 사용하여 list 형식으로 구성된 거래 데이터를 2차원 numpy array로 변경한다.

In [ ]:
te = TransactionEncoder()
te_ary = te.fit_transform(df_tmp)
print(te_ary)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


이 2차원 array를 그대로 dataframe화 한다.

In [ ]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,Bowl Nic Pitt,Bread,Bread Pudding,Brioche and salami,Brownie,Cake,Caramel bites,Cherry me Dried fruit,Chicken Stew,Chicken sand,Chimichurri Oil,Chocolates,Christmas common,Coffee,Coffee granules,Coke,Cookies,Crepes,Crisps,Drinking chocolate spoons,Duck egg,Dulce de Leche,Eggs,Ella's Kitchen Pouches,Empanadas,Extra Salami or Feta,Fairy Doors,Farm House,Focaccia,Frittata,Fudge,Gift voucher,Gingerbread syrup,Granola,Hack the stack,Half slice Monster,Hearty & Seasonal,Honey,Hot chocolate,Jam,Jammie Dodgers,Juice,Keeping It Local,Kids biscuit,Lemon and coconut,Medialuna,Mighty Protein,Mineral water,Mortimer,Muesli,Muffin,My-5 Fruit Shoot,Nomad bag,Olum & polenta,Panatone,Pastry,Pick and Mix Bowls,Pintxos,Polenta,Postcard,Raspberry shortbread sandwich,Raw bars,Salad,Sandwich,Scandinavian,Scone,Siblings,Smoothies,Soup,Spanish Brunch,Spread,Tacos/Fajita,Tartine,Tea,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## Step 4. Association rule mining
- 연관분석 함수(`apriori, association_rules`)를 사용하여 분석 진행  
  - 지지도(support): 범용성 0.1
    - rule의 조건이 일어날 확률
    - 전체 중에서 상품 A가 몇 번이나 판매되었는가  
  $s(X→Y)=P(X \cap Y)$  

  - 신뢰도(confidence) 0.7
    - rule의 조건이 일어났을 때, 결과가 발생할 확률
    - A를 구입한 주문 중 B도 동시에 구입한 주문 수  
  $c(X → Y)=\frac{P(X \cap Y)}{P(X)}=P(B|A)$  

  - 향상도(lift) 
    - 조건과 결과가 각각 독립적이라고 가정할 경우보다 얼마나 더 좋은지를 평가
    - 두 상품의 관계를 직접적으로 나타낸 것  
   $Lift(X, Y)=\frac{P(A \cap B)}{P(A) \times P(B)}=\frac{P(B|A)}{P(B)}$  
    - X가 기본 아이템일 경우 항상 confidence가 1이 나오는 경우 두 아이템 간 관계를 파악할 수 없으므로 향상도를 비교

- 일반적으로 association rule을 사용하여 분석할 때는 다음과 같은 과정을 거침  
    1) 특정 지지도 이상인 set을 찾음  
    2) 해당 set의 조합으로 만들 수 있는 rule 중 신뢰도 이하의 rule을 screening out   
    3) 향상도를 정렬하여 rule을 평가

- 해석 시 유의할 점
  - item의 선/후 혹은 인과 관계는 알 수 없음
  - item별 수량은 고려되지 않음

지지도는 rule의 조건이 일어날 확률로전체 중에서 상품 A가 몇 번이나 판매되었는가를 의미한다. 유의미한 지지도를 가진  상품들을 알기위해  apriori 알고리즘을 활용하는데, 이는 Minimum support를 지정하여 그 비율 이상의 지지도를 갖는 item을 구하게 되고, 그들의 조합으로 지지도를 다시 한번 계산하게 된다. 그리고 minimum 값을 넘는 조합이 없을 때까지 조합에 포함되는 item 개수를 늘려주게 된다. Minimum support의 지정은 범용적으로 0.1을 사용하지만 경우에 따라 0.05를 사용하기도 한다. 본 실습에서는 0.05를 기준으로 지정하여 다음과정을 진행하였다

In [ ]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True) # 0.05를 기준으로 선정
frequent_itemsets.sort_values(by='support', ascending=False)

,support,itemsets
2,0.478394,(Coffee)
0,0.327205,(Bread)
8,0.142631,(Tea)
1,0.103856,(Cake)
9,0.090016,"(Coffee, Bread)"
6,0.086107,(Pastry)
7,0.071844,(Sandwich)
5,0.061807,(Medialuna)
4,0.058320,(Hot chocolate)
10,0.054728,"(Cake, Coffee)"


 신뢰도는 rule의 조건이 일어났을 때, 결과가 발생할 확률로 A를 구입한 주문 중 B도 동시에 구입한 주문 수를 의미한다. 앞서 많이 구매된 메뉴를 사용해 함께 구매된 item들을 확인한다. 컬럼의 antecedents는 조건절, consequents는 결과절을 의미한다. 즉,  Antecedents를 구매했을 때 consequents를 구매한다는 의미로 해석할 수 있다.

In [ ]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False) # 최소 confidence를 0.1로 지정

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
1,(Bread),(Coffee),0.327205,0.478394,0.090016,0.275105,0.575059,-0.066517,0.719561
0,(Coffee),(Bread),0.478394,0.327205,0.090016,0.188163,0.575059,-0.066517,0.828731
3,(Coffee),(Cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905


향상도는 조건과 결과가 각각 독립적이라고 가정할 경우보다 얼마나 더 좋은지를 평가하는 수치로두 상품의 관계를 직접적으로 나타낸 것이다. lift가 1보다 클 경우 A를 샀을 때 B을 살 확률이 높아진다는 것을 의미하고, 1보다 작을 경우 A를 샀을 때 B을 사지 않을 확률이 높은 것을 의미한다. 즉, 1을 기준으로 발견된 연관규칙을 평가한다.

In [ ]:
association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
1,(Coffee),(Cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905


- 지지도, 신뢰도 등의 minimum threshold를 결정할 때 주관성이 들어갈 수 있음
- 목적에 따라 중요하게 판단할 기준이 다를 수 있음
  - 신뢰도가 동일하다고 했을 때, 소비자노출이 중요하다면 지지도가 높은 Rule이, 노출 대비 소비자 반응률(구매여부)이 중요한 경우 향상도가 높은 Rule이 더 좋은 규칙

# Appendices1 : 오전/오후 시간대의 비교

데이터의 period_day컬럼은 transaction이 발생한 시간대에 대한 정보를 포함한다. 이 컬럼의 unique값은 'morning', 'afternoon', 'evening', 'night' 으로 확인할 수 있었다. 이를 오전과 오후 시간대로 나누어 연관 규칙 분석을 진행하였을때 유의미한 차이가 있을 것이라고 생각하여 데이터를 나누어 연관규칙 분석을 진행하였다. group1은 morning을 제외한 나머지 시간대, 즉 오후에 발생한 거래들을 포함하며, group2는 morning, 즉 오전에 발생한 거래들을 포함한다. 

In [ ]:
bread.period_day.unique()

array(['morning', 'afternoon', 'evening', 'night'], dtype=object)

In [ ]:
bread['period_day'].value_counts()

afternoon    11569
morning       8404
evening        520
night           14
Name: period_day, dtype: int64

In [ ]:
data1 = bread[bread['period_day']!='morning']
data2 = bread[bread['period_day']=='morning']

위에서 진행했던 것과 마찬가지로 전처리 과정을 진행한다.

In [ ]:
#1
df_tmp1 = data1.groupby('Transaction')['Item'].apply(set).apply(list).to_list()
print(df_tmp[:10])

df_tmp2 = data2.groupby('Transaction')['Item'].apply(set).apply(list).to_list()
print(df_tmp[:10])

[['Bread'], ['Scandinavian'], ['Cookies', 'Jam', 'Hot chocolate'], ['Muffin'], ['Bread', 'Pastry', 'Coffee'], ['Muffin', 'Pastry', 'Medialuna'], ['Coffee', 'Pastry', 'Medialuna', 'Tea'], ['Pastry', 'Bread'], ['Muffin', 'Bread'], ['Scandinavian', 'Medialuna']]
[['Bread'], ['Scandinavian'], ['Cookies', 'Jam', 'Hot chocolate'], ['Muffin'], ['Bread', 'Pastry', 'Coffee'], ['Muffin', 'Pastry', 'Medialuna'], ['Coffee', 'Pastry', 'Medialuna', 'Tea'], ['Pastry', 'Bread'], ['Muffin', 'Bread'], ['Scandinavian', 'Medialuna']]


In [ ]:
#2
te1 = TransactionEncoder()
te_ary1 = te1.fit_transform(df_tmp1) # df_tmp1를 장바구니 형태로 변경
print(te_ary1)
te2 = TransactionEncoder()
te_ary2 = te2.fit_transform(df_tmp2) # df_tmp2를 장바구니 형태로 변경
print(te_ary2)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [ ]:
#3
df1 = pd.DataFrame(te_ary1, columns=te1.columns_) # label 정보는 te.columns_에 저장되어 있으므로, 해당 값을 칼럼명으로 하는 데이터 프레임으로 변경.
df1.head()
df2 = pd.DataFrame(te_ary2, columns=te2.columns_) # label 정보는 te.columns_에 저장되어 있으므로, 해당 값을 칼럼명으로 하는 데이터 프레임으로 변경.
df2.head()

,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Baguette,Bakewell,Basket,Bread,Brioche and salami,Brownie,Cake,Caramel bites,Cherry me Dried fruit,Chicken Stew,Chocolates,Christmas common,Coffee,Coffee granules,Coke,Cookies,Crisps,Drinking chocolate spoons,Duck egg,Dulce de Leche,Eggs,Ella's Kitchen Pouches,Empanadas,Extra Salami or Feta,Farm House,Focaccia,Frittata,Fudge,Gift voucher,Gingerbread syrup,Granola,Half slice Monster,Hearty & Seasonal,Honey,Hot chocolate,Jam,Jammie Dodgers,Juice,Keeping It Local,Kids biscuit,Lemon and coconut,Medialuna,Mighty Protein,Mineral water,Mortimer,Muesli,Muffin,My-5 Fruit Shoot,Nomad bag,Panatone,Pastry,Raspberry shortbread sandwich,Salad,Sandwich,Scandinavian,Scone,Siblings,Smoothies,Soup,Spanish Brunch,Tacos/Fajita,Tartine,Tea,The BART,The Nomad,Tiffin,Toast,Truffles,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Support 0.05를 기준으로 오전,오후에 대한 분석을 진행한다.

In [ ]:
#1. Support
#group1 : 오후
frequent_itemsets1 = apriori(df1, min_support=0.05, use_colnames=True) # 여기서는 0.05를 기준으로 선정
display(frequent_itemsets1.sort_values(by='support', ascending=False))

,support,itemsets
2,0.450392,(Coffee)
0,0.299702,(Bread)
7,0.169526,(Tea)
1,0.135024,(Cake)
5,0.110780,(Sandwich)
8,0.086908,"(Coffee, Bread)"
9,0.071242,"(Cake, Coffee)"
4,0.062477,(Hot chocolate)
3,0.059679,(Cookies)
10,0.059493,"(Sandwich, Coffee)"


In [ ]:
#1. Support
#group2 : 오전
frequent_itemsets2 = apriori(df2, min_support=0.05, use_colnames=True) # 여기서는 0.05를 기준으로 선정
display(frequent_itemsets2.sort_values(by='support', ascending=False))

,support,itemsets
2,0.514989,(Coffee)
0,0.363149,(Bread)
6,0.139410,(Pastry)
7,0.107482,(Tea)
8,0.094078,"(Bread, Coffee)"
5,0.092615,(Medialuna)
10,0.077261,"(Pastry, Coffee)"
1,0.063125,(Cake)
9,0.054594,"(Coffee, Medialuna)"
4,0.052888,(Hot chocolate)


Confidence 0.1을 기준으로 오전,오후에 대한 분석을 진행한다.

In [ ]:
#2. Confidence
#group1 : 오후
association_rules(frequent_itemsets1, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False) # 최소 confidence를 0.1로 지정

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Sandwich),(Coffee),0.110780,0.450392,0.059493,0.537037,1.192378,0.009599,1.187154
2,(Cake),(Coffee),0.135024,0.450392,0.071242,0.527624,1.171479,0.010428,1.163498
7,(Tea),(Coffee),0.169526,0.450392,0.056695,0.334433,0.742539,-0.019658,0.825775
1,(Bread),(Coffee),0.299702,0.450392,0.086908,0.289981,0.643843,-0.048075,0.774076
0,(Coffee),(Bread),0.450392,0.299702,0.086908,0.192961,0.643843,-0.048075,0.867738
3,(Coffee),(Cake),0.450392,0.135024,0.071242,0.158178,1.171479,0.010428,1.027504
5,(Coffee),(Sandwich),0.450392,0.110780,0.059493,0.132091,1.192378,0.009599,1.024555
6,(Coffee),(Tea),0.450392,0.169526,0.056695,0.125880,0.742539,-0.019658,0.950068


In [ ]:
#2. Confidence
#group2 : 오전
association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False) # 최소 confidence를 0.1로 지정

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(Medialuna),(Coffee),0.092615,0.514989,0.054594,0.589474,1.144633,0.006898,1.181437
4,(Pastry),(Coffee),0.139410,0.514989,0.077261,0.554196,1.076131,0.005466,1.087946
0,(Bread),(Coffee),0.363149,0.514989,0.094078,0.259060,0.503041,-0.092940,0.654589
1,(Coffee),(Bread),0.514989,0.363149,0.094078,0.182679,0.503041,-0.092940,0.779193
5,(Coffee),(Pastry),0.514989,0.139410,0.077261,0.150024,1.076131,0.005466,1.012487
2,(Coffee),(Medialuna),0.514989,0.092615,0.054594,0.106010,1.144633,0.006898,1.014984


Lift 1을 기준으로 오전,오후에 대한 분석을 진행한다.

In [ ]:
#3. Lift
#group1 : 오후
association_rules(frequent_itemsets1, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.103856,0.478394,0.054728,0.526958,1.101515,0.005044,1.102664
1,(Coffee),(Cake),0.478394,0.103856,0.054728,0.114399,1.101515,0.005044,1.011905


In [ ]:
#3. Lift
#group2 : 오전
association_rules(frequent_itemsets2, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Medialuna),(Coffee),0.092615,0.514989,0.054594,0.589474,1.144633,0.006898,1.181437
0,(Coffee),(Medialuna),0.514989,0.092615,0.054594,0.106010,1.144633,0.006898,1.014984
3,(Coffee),(Pastry),0.514989,0.139410,0.077261,0.150024,1.076131,0.005466,1.012487
2,(Pastry),(Coffee),0.139410,0.514989,0.077261,0.554196,1.076131,0.005466,1.087946
